In [1]:
import cv2

In [2]:
pip install --upgrade opencv-python

Note: you may need to restart the kernel to use updated packages.


# Install CUDA

Steps to Install CUDA toolkit from the Ubuntu repository ?  
1. Update system packages

To do this, run the below command:  
$ sudo apt update
2. Install CUDA toolkit 

Now we start installing the CUDA toolkit by apt command:  
$ sudo apt install nvidia-cuda-toolkit


How to Add CUDA to system PATH ?  
Now that we have successfully installed CUDA on your Linux system, we must add CUDA to PATH, so that the shell knows where CUDA is located.  
Let's open the .profile file by vim command:  

$ sudo vim ~/.profile
Then add the following lines to the end of the file and save:

## set PATH for cuda 10.1 installation  
if [ -d "/usr/local/cuda-10.1/bin/" ]; then
export PATH=/usr/local/cuda-10.1/bin${PATH:+:${PATH}}
export LD_LIBRARY_PATH=/usr/local/cuda-10.1/lib64${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}
fi
After saving this file, reboot the system with the below command:  

$ sudo reboot

## Install on UBUNTU

To install CUDA on Ubuntu,  execute the following commands:

$ sudo apt update
$ sudo apt install nvidia-cuda-toolkit
To Check your CUDA version, run the below command:

$ nvcc --version

## How to Check your CUDA version ?
To do this, simply execute the below command:

$ nvcc --version

# Download & compile the YOLO Darknet network

In [3]:
%pwd

'/home/benoit/git/yolo'

5. Download darknet from https://github.com/pjreddie/darknet  

This command will create a new ./darknet folder with all the darket sources & config files (do that once !)

In [4]:
#!git clone https://github.com/pjreddie/darknet
!git clone https://github.com/AlexeyAB/darknet

fatal: le chemin de destination 'darknet' existe déjà et n'est pas un répertoire vide.


6. Open the makefile file and change these line  
GPU=1  
CUDNN=0  
OPENCV=1  

7. Now we need to compile the network in colab

In [5]:
%pwd

'/home/benoit/git/yolo'

In [4]:
%cd ./darknet

/home/benoit/git/yolo/darknet


In [5]:
!chmod +rwx ./*

Build the netword (do that once!)

In [7]:
!make

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
 1147 |             if (iteration_old == 0)
      |             ^~
./src/image_opencv.cpp:1150:10: note: ...this statement, bu

7. Copy the YOLO v4 config & weights file into toe /cfg folder

8. Download the pre-trained weights for the training (https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137)

# Build the Network Config file

1. Copy the yolov4.cfg file an create this new copy: document_yolov4.cfg
2. Open and modify the file with these entries  
Line 3 : subdivisions=12  
Line 7: width=416  
Line 8: height=416  
Line 19: max_batches = 2000  
Line 21: steps=1600,1800  
Line 961,1049,1137: filters=18 # (classes+5)*3  
Line 968,1056,1144: classes=1



# Test the YOLO network

In [16]:
%pwd

'/home/benoit/git/yolo/darknet'

In [18]:
!./darknet detector test cfg/coco.data ../oc_data/yolov4.weights yoloimg.jpg

 CUDA-version: 11000 (11000), GPU count: 1  
 OpenCV version: 4.2.0


# Prepare the custom model

1. Get images from any sources ...
2. Label images by using for example LabelImg (https://github.com/tzutalin/labelImg)

After labeling all your images (here i'll label only 300 images) you should have 3 kind of files:
* the images itself (jpg, png, etc.)
* a set of text file (one per image) associated with the coordinates of the boxes in the image (thes files are generated by LabelImg)
* a classes file which list the classes

Here the images are located in /content/drive/MyDrive/Colab Notebooks/YOLO/darknet/doc_data/doc_images

# Configuration files

## Split the dataset (Train/Test)

1. Create one file with the list of 80% of the images (train) : 240 images in our case (document_training.txt)
2. Create one file with the list of 20% of the images (test) : 60 images in our case (document_test.txt)
3. Create a folder named doc_data into the root of darknet (/content/drive/MyDrive/Colab Notebooks/YOLO/darknet/doc_data)
4. Put the two files in this directory

*Note: the list of image file must repect the correct path (starting into the darknet folder)*

In [ ]:
from os import listdir
from os.path import isfile, join
mydirectory = "/content/drive/MyDrive/Colab Notebooks/YOLO/darknet/doc_data/doc_images"
allfiles = [f for f in listdir(mydirectory) if isfile(join(mydirectory, f))]

List all files in the directory (images + labels + classes)

In [ ]:
len(allfiles)

589

Create the two files (train + test) by splitting per 80%-20% (do that once !)

In [ ]:
rootdir = "doc_data/doc_images/"
counter = 1
images_files = []

# put all the images files into a list
for file in allfiles:
  if (file[len(file)-3:len(file)] != "txt"):
    images_files.append(rootdir +  file)

# Set the size of the test (20%) & train data (80%)
training_limit = int(len(images_files) * 80 / 100)
test_limit = len(images_files) - training_limit

# Build the two lists
images_training_files = []
images_test_files = []
for file in images_files:
  if (counter <= training_limit):
    images_training_files.append(file)
  else:
    images_test_files.append( file)
  counter = counter + 1

# Put the two list into two files
with open('/content/drive/MyDrive/Colab Notebooks/YOLO/darknet/doc_data/document_test.txt', 'w') as f:
    for item in images_test_files:
        f.write("%s\n" % item)

with open('/content/drive/MyDrive/Colab Notebooks/YOLO/darknet/doc_data/document_training.txt', 'w') as f:
    for item in images_training_files:
        f.write("%s\n" % item)

## List of labels/classes

7. Then create a file named **document.names** and put the label (only one here) in there.

## Configuration bundle file

Create a file named **document.data** which will regroup all the configuration. Put that content inside :

classes = 1  
valid = doc_data/document_test.txt  
train = doc_data/document_training.txt  
names = doc_data/document.names  
backup = backup  

# Train the model

In [4]:
%cd ./darknet

/home/benoit/git/yolo/darknet


In [5]:
!chmod +rwx ./*

**In case of problem ... just launch this command line below to restart from the last backup (not from the beggining).**

In [14]:
!./darknet detector train doc_data/document.data doc_data/document_yolov4.cfg doc_data/yolov4.conv.137 -dont_show -map

 CUDA-version: 11000 (11000), GPU count: 1  
 OpenCV version: 4.2.0
 Prepare additional network for mAP calculation...
 0 : compute_capability = 300, cudnn_half = 0, GPU: Quadro K2100M 
net.optimized_memory = 0 
mini_batch = 1, batch = 32, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
conv     32       3 x 3/ 1    224 x 224 x   3 ->  224 x 224 x  32 0.087 BF
   1 conv     64       3 x 3/ 2    224 x 224 x  32 ->  112 x 112 x  64 0.462 BF
   2 conv     64       1 x 1/ 1    112 x 112 x  64 ->  112 x 112 x  64 0.103 BF
   3 route  1 		                           ->  112 x 112 x  64 
   4 conv     64       1 x 1/ 1    112 x 112 x  64 ->  112 x 112 x  64 0.103 BF
   5 conv     32       1 x 1/ 1    112 x 112 x  64 ->  112 x 112 x  32 0.051 BF
   6 conv     64       3 x 3/ 1    112 x 112 x  32 ->  112 x 112 x  64 0.462 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 112 x 112 x  64 0.001 BF
   8 conv     64      

In [11]:
!./darknet detector train doc_data/document.data doc_data/document_yolov4.cfg backup/document_yolov4_last.weights -dont_show -map

 CUDA-version: 11000 (11000), GPU count: 1  
 OpenCV version: 4.2.0
 Prepare additional network for mAP calculation...
 0 : compute_capability = 300, cudnn_half = 0, GPU: Quadro K2100M 
net.optimized_memory = 0 
mini_batch = 1, batch = 32, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
conv     32       3 x 3/ 1    220 x 220 x   3 ->  220 x 220 x  32 0.084 BF
   1 conv     64       3 x 3/ 2    220 x 220 x  32 ->  110 x 110 x  64 0.446 BF
   2 conv     64       1 x 1/ 1    110 x 110 x  64 ->  110 x 110 x  64 0.099 BF
   3 route  1 		                           ->  110 x 110 x  64 
   4 conv     64       1 x 1/ 1    110 x 110 x  64 ->  110 x 110 x  64 0.099 BF
   5 conv     32       1 x 1/ 1    110 x 110 x  64 ->  110 x 110 x  32 0.050 BF
   6 conv     64       3 x 3/ 1    110 x 110 x  32 ->  110 x 110 x  64 0.446 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 110 x 110 x  64 0.001 BF
   8 conv     64      